In [7]:
import pandas as pd
import numpy as np
import os
print(os.environ['IEX_TOKEN'])

pk_2f841cbff2d6422ab0145ada59fa5d34


In [220]:
# Finance data time
from datetime import datetime
from iexfinance.stocks import get_historical_data

start = datetime(2019, 1, 1)
end = datetime(2019, 6, 15)
pd.options.display.float_format = '{:,.2f}'.format
df = pd.DataFrame(get_historical_data("TSLA", start, end, close_only=True)).transpose()

In [221]:
def pos_streaker(x):
    global count
    if(x > 0):
        count += 1
    else:
        count = 0
    return count

In [222]:
def neg_streaker(x):
    global count
    if(x <= 0):
        count += 1
    else:
        count = 0
    return count

In [223]:
df['log_returns'] = np.log(df['close']/df['close'].shift(1))
day_series = pd.date_range(start, end, freq='D').to_series()
day_series_dow = day_series.dt.dayofweek
day_series_dow[:5]

2019-01-01    1
2019-01-02    2
2019-01-03    3
2019-01-04    4
2019-01-05    5
Freq: D, dtype: int64

In [224]:
df['ups'] = np.where(df['log_returns'] > 0, 1, 0)
df['downs'] = np.where(df['log_returns'] <= 0, 1, 0)
df['date'] = df.index

In [225]:
count = 0 
df['up_streaks'] = df['log_returns'].apply(pos_streaker)
df['down_streaks'] = df['log_returns'].apply(neg_streaker)
df['dow'] = pd.to_datetime(df['date'], errors='coerce').dt.day_name()

In [226]:
df[:20]

,close,volume,log_returns,ups,downs,date,up_streaks,down_streaks,dow
2019-01-02,310.12,"11,658,648.00",nan,0,0,2019-01-02,0,0,Wednesday
2019-01-03,300.36,"6,965,184.00",-0.03,0,1,2019-01-03,0,1,Thursday
2019-01-04,317.69,"7,394,116.00",0.06,1,0,2019-01-04,1,0,Friday
2019-01-07,334.96,"7,551,225.00",0.05,1,0,2019-01-07,2,0,Monday
2019-01-08,335.35,"7,008,516.00",0.00,1,0,2019-01-08,3,0,Tuesday
2019-01-09,338.53,"5,432,945.00",0.01,1,0,2019-01-09,4,0,Wednesday
2019-01-10,344.97,"6,056,354.00",0.02,1,0,2019-01-10,5,0,Thursday
2019-01-11,347.26,"5,039,052.00",0.01,1,0,2019-01-11,6,0,Friday
2019-01-14,334.40,"5,247,284.00",-0.04,0,1,2019-01-14,0,1,Monday
2019-01-15,344.43,"6,056,590.00",0.03,1,0,2019-01-15,1,0,Tuesday


In [227]:
up_counts = df['up_streaks'].value_counts(normalize=True)
up_counts

0   0.53
1   0.24
2   0.13
3   0.06
5   0.02
4   0.02
6   0.01
Name: up_streaks, dtype: float64

In [228]:
down_counts = df['down_streaks'].value_counts(normalize=True)
down_counts

0   0.48
1   0.24
2   0.13
3   0.10
4   0.03
5   0.02
6   0.01
Name: down_streaks, dtype: float64

In [ ]:
# TO DO next- drop and group to just Fridays to Mondays